In [11]:
import requests
import json
import pandas as pd
import numpy as np

# Step 1: downloading the data
To get the data, we use the [GDC API](https://docs.gdc.cancer.gov/API/Users_Guide/). Please refer to the link for additional information. 

In [76]:
#where do we ask the api
cases_endpt = 'https://api.gdc.cancer.gov/cases'

#how to filter our cases
filt = {"op":"=",
        "content":{
            "field": "primary_site",
            "value": ["breast"]
        }
}

#the actual request.
params = {'filters':json.dumps(filt),
          'format': "tsv",
          'fields':"case_id",
          'expand':"demographic,diagnoses,exposures,diagnoses.treatments", 
          # returns all the fields in the demographic group, etc. 
            'size':10000} #set size to 10K to get all of the ~9K samples

In [77]:
#note: idkwy, but the request takes some time with pytohn.
resp = requests.post(cases_endpt, json = params)

In [78]:
with open("clinical.tsv", 'w') as f:
    f.write(resp.text)

In [79]:
clinical_data = pd.read_csv("clinical.tsv", sep="\t", low_memory=False)

In [80]:
list(clinical_data.columns)

['case_id',
 'demographic.age_at_index',
 'demographic.age_is_obfuscated',
 'demographic.cause_of_death',
 'demographic.cause_of_death_source',
 'demographic.country_of_residence_at_enrollment',
 'demographic.created_datetime',
 'demographic.days_to_birth',
 'demographic.days_to_death',
 'demographic.demographic_id',
 'demographic.ethnicity',
 'demographic.gender',
 'demographic.occupation_duration_years',
 'demographic.premature_at_birth',
 'demographic.race',
 'demographic.state',
 'demographic.submitter_id',
 'demographic.updated_datetime',
 'demographic.vital_status',
 'demographic.weeks_gestation_at_birth',
 'demographic.year_of_birth',
 'demographic.year_of_death',
 'diagnoses.0.age_at_diagnosis',
 'diagnoses.0.ajcc_clinical_m',
 'diagnoses.0.ajcc_clinical_n',
 'diagnoses.0.ajcc_clinical_stage',
 'diagnoses.0.ajcc_clinical_t',
 'diagnoses.0.ajcc_pathologic_m',
 'diagnoses.0.ajcc_pathologic_n',
 'diagnoses.0.ajcc_pathologic_stage',
 'diagnoses.0.ajcc_pathologic_t',
 'diagnoses.0.a

Sadly most of the data is a nan...